# Data Formatter for Database schema

Data formatter script to fit data into Database.

## Dependencies

In [3]:
import json
import pandas as pd

## File Paths

In [8]:
awisMetricsFilePath = './blogsAWISMetricsDbInsertion.json'
scrapedDataFilePath = './blogDataDbInsertion.json'
bloggerDomainsFilePath = '.\Bloggers domain WIP.csv'
tempBlogsAWISMetricsFilePath = './tempBlogsAWISMetricsDbInsertion.json'

## Data Format for AWIS and Blogs metrics

In [9]:
with open(awisMetricsFilePath) as json_file:
    awisMetrics = json.load(json_file)
    
with open(scrapedDataFilePath) as json_file:
    scrapedData = json.load(json_file)
    
allDomainNames = list(pd.read_csv(bloggerDomainsFilePath).domain_name)

with open(tempBlogsAWISMetricsFilePath) as json_file:
    tempBlogsAWISMetrics = json.load(json_file)

In [4]:
len(awisMetrics.get("blogs"))

205

In [5]:
len(scrapedData.get('blogs'))

180

In [6]:
scrapedData.get('blogs')[2].get("blogUrl")

'littlegreybox.net'

In [7]:
count = 0
numUrlNotFound = 0
flag = 0
tempList = []

for awisDetails in awisMetrics.get("blogs"):
    for blogDetails in scrapedData.get('blogs'):
        if awisDetails.get('blogUrl').split('.')[0] in blogDetails.get('blogUrl'):
            tempDict = blogDetails
            tempDict.update({'blogTraffic': awisDetails.get('blogTraffic')})
            tempList.append(tempDict)
            count = count + 1
            flag = 1
            break
    if not flag:
        numUrlNotFound = numUrlNotFound + 1
#         print("url not found:", awisDetails.get('blogUrl'))
    flag = 0
blogs = {'blogs': tempList}

print("Number of Blog Domains with scraped data:", count)
print("Number of Blog Domains without scraped data:", numUrlNotFound)

Number of Blog Domains with scraped data: 171
Number of Blog Domains without scraped data: 34


In [8]:
len(tempList)

171

In [9]:
# with open('./finalAwisAndBlogsDataDbInsertion.json', 'w') as outfile:
#     json.dump(blogs, outfile)

## Data Format for Entity visualisation (Rach)

In [10]:
scrapedData.get('blogs')[0].get('blogs')[0].keys()

dict_keys(['blogPostUrl', 'blogPost', 'locations', 'spacyNer', 'awsNer', 'writtenAt'])

In [11]:
scrapedData.get('blogs')[0].get('blogs')[0].get('locations')

{'country': 'Thailand',
 'cities': '',
 'placesOfInterest': [['Chinese Temple',
   'Cultural, Attractions',
   9.577891699999999,
   99.97393690000001]]}

In [12]:
# Retrieve Entity Country mapping
with open('entitiesCountryDict.json', 'r') as f:
    entitiesCountryDict = json.load(f)

In [13]:
fullEntitiesList = []
fullCountryList = []
fullBlogPostDateList = []

for blog in scrapedData.get('blogs'):
    for blogPost in blog.get('blogs'):
        blogPostEntitiesList = []
        countryList = []
        locations = blogPost.get('locations')
        
        # Process POIs
        placesOfInterest = locations.get('placesOfInterest')
        for poiData in placesOfInterest:
            poi = poiData[0]
            blogPostEntitiesList.append(poi)
    
            # Process Country
            country = entitiesCountryDict.get(poi)
            countryList.append(country)
#             countryList = countryList*len(blogPostEntitiesList)
            
        # Process Country
#         countryList = [str(locations.get('country').split(','))]
#         countryList = countryList*len(blogPostEntitiesList)
        
        # Process Date
        blogPostDateList = [blogPost.get('writtenAt')]
        blogPostDateList = blogPostDateList*len(blogPostEntitiesList)                
        
        # Append to full lists
        fullEntitiesList.extend(blogPostEntitiesList)
        fullCountryList.extend(countryList)
        fullBlogPostDateList.extend(blogPostDateList)
        
# print(fullEntitiesList)
# print(fullCountryList)
# print(fullBlogPostDateList)

In [14]:
print(len(fullCountryList))
print(len(fullEntitiesList))
print(len(fullBlogPostDateList))

1273
1273
1273


In [15]:
dateEntitiesDf = pd.DataFrame({'writtenAt': fullBlogPostDateList, 'country': fullCountryList, 'entities': fullEntitiesList}).sort_values(by='writtenAt')

In [16]:
print('Total number of rows:', len(dateEntitiesDf))
print('Number of rows without date:', len(dateEntitiesDf[dateEntitiesDf['writtenAt'] == '']))

# Remove rows without date
cleanedDateEntitiesDf = dateEntitiesDf[dateEntitiesDf['writtenAt'] != '']
print('Number of rows with date:', len(cleanedDateEntitiesDf))

Total number of rows: 1273
Number of rows without date: 181
Number of rows with date: 1092


In [17]:
monthMapping = {
    '01': 'Jan',
    '02': 'Feb',
    '03': 'Mar',
    '04': 'Apr',
    '05': 'May',
    '06': 'Jun',
    '07': 'Jul',
    '08': 'Aug',
    '09': 'Sep',
    '10': 'Oct',
    '11': 'Nov',
    '12': 'Dec'
}

In [18]:
def getMonth(date):
    return monthMapping.get(date.split('-')[1]) if len(date.split('-')) > 0 else ''

def getYear(date):
    return date.split('-')[0] if len(date.split('-')) > 0 else ''

# cleanedDateEntitiesDf['writtenAt'].apply(lambda x: getMonth(x))
cleanedDateEntitiesDf['writtenAtMth'] = cleanedDateEntitiesDf['writtenAt'].apply(lambda x: getMonth(x))
cleanedDateEntitiesDf['writtenAtYear'] = cleanedDateEntitiesDf['writtenAt'].apply(lambda x: getYear(x))

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [19]:
groupMthYearDf = cleanedDateEntitiesDf.copy()
groupMthYearDf = groupMthYearDf.groupby(['country','writtenAtMth', 'writtenAtYear', 'entities']).count().reset_index()
groupMthYearDf.rename(columns={'writtenAt': 'count'}, inplace=True)
groupMthYearDf = groupMthYearDf.sort_values(by='count', ascending=False)
print(len(groupMthYearDf))
groupMthYearDf

986


,country,writtenAtMth,writtenAtYear,entities,count
919,Thailand,Jul,2019,Airplane Graveyard,4
200,Japan,Jun,2016,Osaka,3
274,Japan,Sep,2018,Brand New,3
234,Japan,May,2017,Osaka,3
711,Singapore,Oct,2016,Chinatown,3
247,Japan,Nov,2016,Osaka,3
95,Japan,Apr,2017,Osaka,3
471,Singapore,Jan,2018,Orchard Road,3
298,Singapore,Apr,2017,Chinatown,3
319,Singapore,Apr,2017,Woodlands,3


In [20]:
groupMthYearDf[groupMthYearDf['entities'] == 'Airplane Graveyard']

,country,writtenAtMth,writtenAtYear,entities,count
919,Thailand,Jul,2019,Airplane Graveyard,4
897,Thailand,Dec,2018,Airplane Graveyard,1
885,Thailand,Apr,2019,Airplane Graveyard,1
927,Thailand,Jun,2019,Airplane Graveyard,1
952,Thailand,Nov,2018,Airplane Graveyard,1


In [45]:
groupMthYearDf[groupMthYearDf['country'] == 'Thailand'].groupby(['entities', 'writtenAtYear']).sum().sort_values(by='count', ascending=False)

count
entities                writtenAtYear       
Lagoon                  2017               6
                        2016               6
Airplane Graveyard      2019               6
Lagoon                  2018               5
                        2014               4
Mai Khao Beach          2018               4
FAST TRACK              2018               3
Lagoon                  2019               3
Mai Khao Beach          2016               2
                        2017               2
Giant Swing             2018               2
Malibu beach            2016               2
Seafood Market          2018               2
Sukhumvit               2012               2
                        2014               2
The Grand Palace        2018               2
Lagoon                  2012               2
Airplane Graveyard      2018               2
China Town              2017               2
                        2018               2
Chao Phraya River       2018               2
FAST TRACK              2017               2
Freedom Beach           2016               1
Patong Beach            2009               1
                        2017               1
Railay Beach            2011               1
                        2018               1
Safari World            2012               1
Sathorn Pier            2012               1
Seafood Market          2013               1
...                                      ...
Wat Pathum Wanaram      2017               1
Wat Phra Singh          2015               1
Patong Beach            2015               1
Chiang Mai Zoo          2015               1
Chaweng Beach           2019               1
Freedom Beach           2018               1
Funarium                2018               1
Fantasia Lagoon         2013               1
Giant Swing             2019               1
Kamala Beach            2019               1
Karon Beach             2017               1
Kata Beach              2013               1
Khao San Road           2009               1
                        2010               1
                        2016               1
                        2018               1
Lagoon                  2010               1
                        2011               1
FAST TRACK              2019               1
Lagoon                  2013               1
                        2015               1
FAST TRACK              2015               1
Coral Island            2018               1
Chinese Temple          2016               1
Lamai Beach             2018               1
Lumpini Park            2018               1
Madame Tussauds Bangkok 2012               1
Mai Khao Beach          2015               1
China Town              2016               1
Windmill Viewpoint      2015               1

[75 rows x 1 columns]

In [40]:
groupMthYearDf[groupMthYearDf['country'] == 'South Korea'].groupby('entities').sum().sort_values(by='count', ascending=False)

,count
entities,
Lotte World,9
Sinchon,7
Haebangchon,6
Itaewon,6
The Gallery,3
The Arc,2
Cheongnaho Lake,1
Little Tokyo,1
Starfield Library,1


In [ ]:
groupMthYearDf[groupMthYearDf['country'] == 'Singapore'].groupby('entities').sum().sort_values(by='count', ascending=False)

In [23]:
dateEntitiesDict = {}
for index, row in groupMthYearDf.iterrows():
#     dateEntitiesDict.update({row['writtenAt']: {row['entities']: row['count']}})

    country = row['country']
    year = row['writtenAtYear']
    mth = row['writtenAtMth']
    entity = row['entities']
    entityCount = row['count']
    
    
    if country not in dateEntitiesDict.keys():
        dateEntitiesDict.update({
            country: {
                year: {
                    'entities': {
                        entity : [mth, entityCount]
                    }
                }
            }
        })
    elif year not in dateEntitiesDict.get(country).keys():
        dateEntitiesDict.get(country).update({
            year: {
                    'entities': {
                        entity : [mth, entityCount]
                    }
                }
        })
    elif entity not in dateEntitiesDict.get(country).get(year).get('entities').keys():
#         print(entity)
#         print(mth, year)
#         print(dateEntitiesDict.get(country).get(year).get('entities').keys())
#         print('\n')
        dateEntitiesDict.get(country).get(year).get('entities').update({
            entity : [mth, entityCount]
        })
    else:
#         print(entity)
#         print(mth, year)
        entities = dateEntitiesDict.get(country).get(year).get('entities')
        if entities.get(entity):
            updatedEntities = entities.get(entity)
            updatedEntities.append([mth, entityCount])
            entities.update({
                entity : updatedEntities
            })
        else:
            entities.update({
                entity :[mth, entityCount]
            })
# dateEntitiesDict

In [21]:
# cleanedDateEntitiesDf = cleanedDateEntitiesDf.sort_values('writtenAt', ascending=False)
# cleanedDateEntitiesDf

In [22]:
# groupByMthYearDf = cleanedDateEntitiesDf.groupby(['country', 'writtenAtMth', 'writtenAtYear', 'entities']).count().reset_index().sort_values(by='count', ascending=False)
# groupByMthYearDf

In [24]:
# Writes to output json 
# with open('dateEntitiesDbInsertion.json', 'w') as fp:
#     json.dump(dateEntitiesDict, fp)

## Data Format for Word Cloud (Des)

In [25]:
dateOneDayBefore = '2019-09-02'
dateSevenDaysBefore = '2019-08-27'
dateOneMthBefore = '2019-08-03'
dateThreeMthsBefore = '2019-06-02'

In [26]:
oneDayBeforeDf = cleanedDateEntitiesDf[cleanedDateEntitiesDf['writtenAt'] == dateOneDayBefore]
print(len(oneDayBeforeDf))
oneDayBeforeDf.head()

34


,writtenAt,country,entities,writtenAtMth,writtenAtYear
114,2019-09-02,Singapore,River Safari,Sep,2019
107,2019-09-02,Singapore,Asian Civilisations Museum,Sep,2019
112,2019-09-02,Singapore,Gardens by the Bay,Sep,2019
111,2019-09-02,Singapore,Marina Bay,Sep,2019
104,2019-09-02,Singapore,Malay Heritage Centre,Sep,2019


In [27]:
groupByOneDayBeforeEntityDf = oneDayBeforeDf.groupby(['entities']).count().reset_index()
groupByOneDayBeforeEntityDf.rename(columns={'writtenAt': 'count'}, inplace=True)
groupByOneDayBeforeEntityDf = groupByOneDayBeforeEntityDf[['entities', 'count']].sort_values(by='count', ascending=False)
groupByOneDayBeforeEntityDf

# groupByOneDayBeforeEntityDf = oneDayBeforeDf.groupby(['entities']).count().reset_index().sort_values(by='count', ascending=False)
# groupByOneDayBeforeEntityDf = groupByOneDayBeforeEntityDf[['entities', 'count']]
# print(len(groupByOneDayBeforeEntityDf))
# groupByOneDayBeforeEntityDf

,entities,count
0,Asian Civilisations Museum,1
25,Osaka,1
19,Marina Bay,1
20,Maxwell Food Centre,1
21,Merlion Park,1
22,National Museum of Singapore,1
23,Night Safari,1
24,Orchard Road,1
26,River Safari,1
1,Bollywood Veggies,1


In [28]:
sevenDayBeforeDf = cleanedDateEntitiesDf[cleanedDateEntitiesDf['writtenAt'] >= dateSevenDaysBefore]
print(len(sevenDayBeforeDf))
sevenDayBeforeDf.head()

34


,writtenAt,country,entities,writtenAtMth,writtenAtYear
114,2019-09-02,Singapore,River Safari,Sep,2019
107,2019-09-02,Singapore,Asian Civilisations Museum,Sep,2019
112,2019-09-02,Singapore,Gardens by the Bay,Sep,2019
111,2019-09-02,Singapore,Marina Bay,Sep,2019
104,2019-09-02,Singapore,Malay Heritage Centre,Sep,2019


In [29]:
groupBySevenDaysBeforeEntityDf = sevenDayBeforeDf.groupby(['entities']).count().reset_index()
groupBySevenDaysBeforeEntityDf.rename(columns={'writtenAt': 'count'}, inplace=True)
groupBySevenDaysBeforeEntityDf = groupBySevenDaysBeforeEntityDf[['entities', 'count']].sort_values(by='count', ascending=False)
groupBySevenDaysBeforeEntityDf

# groupBySevenDaysBeforeEntityDf = sevenDayBeforeDf.groupby(['entities']).count().reset_index().sort_values(by='count', ascending=False)
# print(len(groupBySevenDaysBeforeEntityDf))
# groupBySevenDaysBeforeEntityDf

,entities,count
0,Asian Civilisations Museum,1
25,Osaka,1
19,Marina Bay,1
20,Maxwell Food Centre,1
21,Merlion Park,1
22,National Museum of Singapore,1
23,Night Safari,1
24,Orchard Road,1
26,River Safari,1
1,Bollywood Veggies,1


In [30]:
oneMthBeforeDf = cleanedDateEntitiesDf[cleanedDateEntitiesDf['writtenAt'] >= dateOneMthBefore]
print(len(oneMthBeforeDf))
oneMthBeforeDf.tail()

40


,writtenAt,country,entities,writtenAtMth,writtenAtYear
121,2019-09-02,Singapore,ION Orchard,Sep,2019
120,2019-09-02,Singapore,Orchard Road,Sep,2019
119,2019-09-02,Singapore,Golden Mile Tower,Sep,2019
117,2019-09-02,Singapore,Jurong Bird Park,Sep,2019
105,2019-09-02,Singapore,Maxwell Food Centre,Sep,2019


In [31]:
groupByOneMthBeforeEntityDf = oneMthBeforeDf.groupby(['entities']).count().reset_index()
groupByOneMthBeforeEntityDf.rename(columns={'writtenAt': 'count'}, inplace=True)
groupByOneMthBeforeEntityDf = groupByOneMthBeforeEntityDf[['entities', 'count']].sort_values(by='count', ascending=False)
groupByOneMthBeforeEntityDf

# groupByOneMthBeforeEntityDf = oneMthBeforeDf.groupby(['entities']).count().reset_index().sort_values(by='count', ascending=False)
# print(len(groupByOneMthBeforeEntityDf))
# groupByOneMthBeforeEntityDf

,entities,count
2,Brand New,3
26,Osaka,3
4,Chinatown,2
0,Asian Civilisations Museum,1
27,River Safari,1
21,Maxwell Food Centre,1
22,Merlion Park,1
23,National Museum of Singapore,1
24,Night Safari,1
25,Orchard Road,1


In [32]:
threeMthsBeforeDf = cleanedDateEntitiesDf[cleanedDateEntitiesDf['writtenAt'] >= dateThreeMthsBefore]
print(len(threeMthsBeforeDf))
threeMthsBeforeDf.head()

67


,writtenAt,country,entities,writtenAtMth,writtenAtYear
1060,2019-06-08,Singapore,Chinatown,Jun,2019
1061,2019-06-08,Singapore,Gardens by the Bay,Jun,2019
1125,2019-06-10,Thailand,Lagoon,Jun,2019
444,2019-06-16,Japan,Colony,Jun,2019
443,2019-06-16,Japan,Brand New,Jun,2019


In [33]:
groupByThreeMthsBeforeEntityDf = threeMthsBeforeDf.groupby(['entities']).count().reset_index()
groupByThreeMthsBeforeEntityDf.rename(columns={'writtenAt': 'count'}, inplace=True)
groupByThreeMthsBeforeEntityDf = groupByThreeMthsBeforeEntityDf[['entities', 'count']].sort_values(by='count', ascending=False)
groupByThreeMthsBeforeEntityDf


# groupByThreeMthsBeforeEntityDf = threeMthsBeforeDf.groupby(['entities']).count().reset_index().sort_values(by='count', ascending=False)
# print(len(groupByThreeMthsBeforeEntityDf))
# groupByThreeMthsBeforeEntityDf

,entities,count
3,Brand New,6
0,Airplane Graveyard,5
6,Chinatown,4
37,Osaka,3
13,Gardens by the Bay,2
43,Tanjong Pagar,2
30,Marina Bay,2
29,Marina Barrage,2
47,Woodlands,2
9,East Coast Park,1


In [34]:
finalTimePeriodEntitiesDict = {}

finalTimePeriodEntitiesDict.update({
                                    'oneDay': {
                                        'entities': dict(zip(groupByOneDayBeforeEntityDf['entities'], groupByOneDayBeforeEntityDf['count']))
                                    }
})

finalTimePeriodEntitiesDict.update({
                    'sevenDays': {
                        'entities': dict(zip(groupBySevenDaysBeforeEntityDf['entities'], groupBySevenDaysBeforeEntityDf['count']))
                    }
})

finalTimePeriodEntitiesDict.update({
                    'oneMth': {
                        'entities': dict(zip(groupByOneMthBeforeEntityDf['entities'], groupByOneMthBeforeEntityDf['count']))
                    }
})

finalTimePeriodEntitiesDict.update({
                    'threeMth': {
                        'entities': dict(zip(groupByThreeMthsBeforeEntityDf['entities'], groupByThreeMthsBeforeEntityDf['count']))
                    }
})

finalTimePeriodEntitiesDict

{'oneDay': {'entities': {'Asian Civilisations Museum': 1,
   'Osaka': 1,
   'Marina Bay': 1,
   'Maxwell Food Centre': 1,
   'Merlion Park': 1,
   'National Museum of Singapore': 1,
   'Night Safari': 1,
   'Orchard Road': 1,
   'River Safari': 1,
   'Bollywood Veggies': 1,
   'Sultan Mosque': 1,
   'Sungei Buloh Wetland Reserve': 1,
   'Supertree Grove': 1,
   'Tanjong Pagar': 1,
   'The Intan': 1,
   'UOB Plaza': 1,
   'Marina Barrage': 1,
   'Malay Heritage Centre': 1,
   'MacRitchie Reservoir': 1,
   'Little India': 1,
   'Kusu Island': 1,
   'Kranji Countryside': 1,
   'Jurong Bird Park': 1,
   'Indian Heritage Centre': 1,
   'ION Orchard': 1,
   'Hay Dairies': 1,
   'Haji Lane': 1,
   'Golden Mile Tower': 1,
   'Giant Swing': 1,
   'Gardens by the Bay': 1,
   'Chinatown': 1,
   'Chijmes': 1,
   'Brand New': 1,
   'Woodlands': 1}},
 'sevenDays': {'entities': {'Asian Civilisations Museum': 1,
   'Osaka': 1,
   'Marina Bay': 1,
   'Maxwell Food Centre': 1,
   'Merlion Park': 1,
   '

In [90]:
# Writes to output json 
# with open('timePeriodEntitiesDbInsertion.json', 'w') as fp:
#     json.dump(finalTimePeriodEntitiesDict, fp)

## Data Format for MedianLoadTime Comparison Chart (Des)

In [55]:
from statistics import median

medianLoadTimeList = []
medianLoadTimeValuesList = []
for blog in awisMetrics.get('blogs'):
    blogUrl = blog.get('blogUrl')
    blogMedianLoadTime = blog.get('blogTraffic').get('contentData').get('speed').get('medianLoadTime')
    if blogMedianLoadTime:
        medianLoadTimeList.append({
            'name': blogUrl,
            'medianLoadTime': blogMedianLoadTime
        }) 
        medianLoadTimeValuesList.append(blogMedianLoadTime)
print(len(medianLoadTimeList))
print(medianLoadTimeList)

92
[{'name': 'loveandroad.com', 'medianLoadTime': 2266}, {'name': 'laughtraveleat.com', 'medianLoadTime': 6593}, {'name': 'littlegreybox.net', 'medianLoadTime': 7185}, {'name': 'mitsueki.sg', 'medianLoadTime': 4872}, {'name': 'thetravelintern.com', 'medianLoadTime': 3143}, {'name': 'belaroundtheworld.com', 'medianLoadTime': 4548}, {'name': '2bearbear.com', 'medianLoadTime': 1880}, {'name': 'flyhoneystars.com', 'medianLoadTime': 8781}, {'name': 'wanderingearl.com', 'medianLoadTime': 2304}, {'name': 'travelinspiration360.com', 'medianLoadTime': 8680}, {'name': 'iwandered.net', 'medianLoadTime': 6277}, {'name': 'wanderlex.com', 'medianLoadTime': 7452}, {'name': 'cheekiemonkie.net', 'medianLoadTime': 4999}, {'name': 'irene-travelogue.com', 'medianLoadTime': 2330}, {'name': 'floraisabelle.com', 'medianLoadTime': 834}, {'name': 'travelerfolio.com', 'medianLoadTime': 3609}, {'name': 'wildjunket.com', 'medianLoadTime': 4900}, {'name': 'sunriseodyssey.com', 'medianLoadTime': 5028}, {'name': 'ha

In [58]:
finalMedianLoadTimeDict = {
    'medianLoadTimeAllBlogs': median(medianLoadTimeValuesList),
    'medianLoadTIme': medianLoadTimeList
}
finalMedianLoadTimeDict

{'medianLoadTimeAllBlogs': 2938.5,
 'medianLoadTIme': [{'name': 'loveandroad.com', 'medianLoadTime': 2266},
  {'name': 'laughtraveleat.com', 'medianLoadTime': 6593},
  {'name': 'littlegreybox.net', 'medianLoadTime': 7185},
  {'name': 'mitsueki.sg', 'medianLoadTime': 4872},
  {'name': 'thetravelintern.com', 'medianLoadTime': 3143},
  {'name': 'belaroundtheworld.com', 'medianLoadTime': 4548},
  {'name': '2bearbear.com', 'medianLoadTime': 1880},
  {'name': 'flyhoneystars.com', 'medianLoadTime': 8781},
  {'name': 'wanderingearl.com', 'medianLoadTime': 2304},
  {'name': 'travelinspiration360.com', 'medianLoadTime': 8680},
  {'name': 'iwandered.net', 'medianLoadTime': 6277},
  {'name': 'wanderlex.com', 'medianLoadTime': 7452},
  {'name': 'cheekiemonkie.net', 'medianLoadTime': 4999},
  {'name': 'irene-travelogue.com', 'medianLoadTime': 2330},
  {'name': 'floraisabelle.com', 'medianLoadTime': 834},
  {'name': 'travelerfolio.com', 'medianLoadTime': 3609},
  {'name': 'wildjunket.com', 'medianLoa

In [59]:
# Writes to output json 
# with open('finalMedianLoadTimeDbInsertion.json', 'w') as fp:
#     json.dump(finalMedianLoadTimeDict, fp)

## Data Format for Average Median Data across all blogs (Des)

In [91]:
medianAllMetricsValuesList = []

for blog in awisMetrics.get('blogs'):
    blogUrl = blog.get('blogUrl')
    usageData = blog.get('blogTraffic').get('usageData')
    if usageData:
        blogThreeMthsUsageData = usageData[0]
    
    trafficRank = blogThreeMthsUsageData.get('rank').get('value')
    reachRank = blogThreeMthsUsageData.get('reach').get('rank').get('value')
    reachPerMillion = blogThreeMthsUsageData.get('reach').get('perMillion').get('value')
    
    PageViewsPerMillion = blogThreeMthsUsageData.get('pageViews').get('perMillion').get('value')
    PageViewsRank = blogThreeMthsUsageData.get('pageViews').get('rank').get('value')
    PageViewsPerUser = blogThreeMthsUsageData.get('pageViews').get('perUser').get('value')
    
    if ',' in str(reachPerMillion):
        reachPerMillion = reachPerMillion.replace(',','')
        print(reachPerMillion)
        reachPerMillion = int(reachPerMillion)
    
    if ',' in str(PageViewsPerMillion):
        PageViewsPerMillion = PageViewsPerMillion.replace(',','')
        print(PageViewsPerMillion)
        PageViewsPerMillion = int(PageViewsPerMillion)
    
    metrics = [trafficRank, reachRank, reachPerMillion, PageViewsPerMillion, PageViewsRank, PageViewsPerUser]
    for idx in range(0, len(metrics)):
        if len(medianAllMetricsValuesList) > idx:
            medianAllMetricsValuesList[idx].append(metrics[idx])  
        else:
            medianAllMetricsValuesList.append([metrics[idx]])
medianAllMetricsValuesList

76740
13568


[[230557,
  954491,
  498332,
  4209032,
  251946,
  1518153,
  127634,
  320052,
  1618472,
  2449731,
  3805995,
  329011,
  5715503,
  2502847,
  547833,
  756495,
  789427,
  2696333,
  1224275,
  1381148,
  190132,
  996850,
  159593,
  159593,
  97581,
  7144084,
  461203,
  461203,
  7981314,
  2825979,
  413882,
  1265463,
  1924764,
  1924764,
  4994823,
  564066,
  564066,
  186120,
  3258628,
  5395615,
  195708,
  5949858,
  2186420,
  193308,
  1677869,
  3722859,
  3030491,
  214022,
  1282215,
  2702542,
  498324,
  498324,
  3370231,
  1051063,
  3926441,
  2665486,
  2150851,
  1455405,
  1455405,
  1221785,
  293275,
  2371323,
  2097077,
  2097077,
  967347,
  536488,
  596802,
  700001,
  1822029,
  1822029,
  1822029,
  7873280,
  3830167,
  1374987,
  1574832,
  1574832,
  1574832,
  1574832,
  2011772,
  8152324,
  8152324,
  8152324,
  193397,
  262301,
  262301,
  311404,
  1018737,
  298359,
  147244,
  259141,
  259141,
  603603,
  708584,
  143621,
  1773001

In [97]:
medianDataList = []
medianMetricsNameList = ['trafficRank', 'reachRank', 'reachPerMillion', 'PageViewsPerMillion', 'PageViewsRank', 'PageViewsPerUser']
medianMetricsValuesList = [median(metricList) for metricList in medianAllMetricsValuesList]
for idx in range(0, len(medianMetricsNameList)):
    medianDataList.append({
        'name': medianMetricsNameList[idx],
        'value': medianMetricsValuesList[idx]
    })
finalMedianMetricsData = {'medianDataAllBlogs': {'threeMths':medianDataList}}
finalMedianMetricsData

{'medianDataAllBlogs': {'threeMths': [{'name': 'trafficRank', 'value': 899927},
   {'name': 'reachRank', 'value': 826718},
   {'name': 'reachPerMillion', 'value': 0.51},
   {'name': 'PageViewsPerMillion', 'value': 0.02},
   {'name': 'PageViewsRank', 'value': 1096781},
   {'name': 'PageViewsPerUser', 'value': 1.6}]}}

In [98]:
# Writes to output json 
# with open('finalMedianMetricsDbInsertion.json', 'w') as fp:
#     json.dump(finalMedianMetricsData, fp)

## Data Generation for all daily historical blog metrics (Des)

In [93]:
from random import uniform, randint

def generateDailyHistoricalData(randomAverage):
    
    historicalDataDict = {}
    rand1, rand2, rand3, rand4 = 0,0,0,0 
    
    while not rand1: 
        rand1 = round(uniform(randomAverage-0.5, randomAverage+0.5),2)
    while not rand2: 
        rand2 = round(uniform(randomAverage-2, randomAverage+0.2),2)
    while not rand3: 
        randomAverageLarge = randomAverage*100000
        rand3 = round(uniform(randomAverageLarge-50000, randomAverageLarge+10000))
    while not rand4: 
        rand4 = round(uniform(randomAverage-0.5, randomAverage+0.5),2)
            
    historicalDataDict = {  
        "pageViews": {
            "perMillion": rand1,
            "perUser": rand2
        },
        "rank": rand3,
        "reach": {
            "perMillion": rand4
        }
    }
    return historicalDataDict
generateDailyHistoricalData(10)

{'pageViews': {'perMillion': 10.17, 'perUser': 9.09},
 'rank': 972003,
 'reach': {'perMillion': 10.35}}

In [94]:
import datetime
# numDays = 31
# startDate = '2019-11-01'
# startDate = datetime.date(int(startDate.split('-')[0]), int(startDate.split('-')[1]), int(startDate.split('-')[2]))
# for timedelta in range(1, numDays+1):  
#     date = startDate - datetime.timedelta(timedelta) 
#     print(date)

# startDate = '2019-11-05' 
def generateTimePeriodHistoricalData(startDate):
    allHistoricalData = []
    startDate = datetime.date(int(startDate.split('-')[0]), int(startDate.split('-')[1]), int(startDate.split('-')[2]))
    randomAverage = randint(1, 10)
    for timedelta in range(1, numDays+1):  
        date = startDate - datetime.timedelta(timedelta) 
        historicalData = generateDailyHistoricalData(randomAverage)
        dateDict = {'date': str(date)}
        dateDict.update(historicalData)
        allHistoricalData.append(dateDict)
#         print(historicalData)
    return {'historicalData': allHistoricalData}
generateTimePeriodHistoricalData('2019-11-12')

{'historicalData': [{'date': '2019-11-11',
   'pageViews': {'perMillion': 10.24, 'perUser': 9.9},
   'rank': 987706,
   'reach': {'perMillion': 10.49}},
  {'date': '2019-11-10',
   'pageViews': {'perMillion': 9.99, 'perUser': 9.74},
   'rank': 1008567,
   'reach': {'perMillion': 10.1}},
  {'date': '2019-11-09',
   'pageViews': {'perMillion': 10.47, 'perUser': 8.15},
   'rank': 951641,
   'reach': {'perMillion': 9.91}},
  {'date': '2019-11-08',
   'pageViews': {'perMillion': 9.99, 'perUser': 8.46},
   'rank': 984320,
   'reach': {'perMillion': 10.39}},
  {'date': '2019-11-07',
   'pageViews': {'perMillion': 10.23, 'perUser': 8.35},
   'rank': 982429,
   'reach': {'perMillion': 10.45}},
  {'date': '2019-11-06',
   'pageViews': {'perMillion': 10.23, 'perUser': 9.4},
   'rank': 1003079,
   'reach': {'perMillion': 10.01}},
  {'date': '2019-11-05',
   'pageViews': {'perMillion': 9.51, 'perUser': 9.03},
   'rank': 1009716,
   'reach': {'perMillion': 9.56}},
  {'date': '2019-11-04',
   'pageVi

In [95]:
for blog in tempBlogsAWISMetrics.get('blogs'):
    blog.get('blogTraffic').update(generateTimePeriodHistoricalData('2019-11-12'))

In [96]:
# Writes to output json 
# with open('finalBlogsAWISMetricsDbInsertion.json', 'w') as fp:
#     json.dump(tempBlogsAWISMetrics, fp)